In [14]:
import copy

import pandas as pd
import numpy as np
import tensorflow

from keras import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split

# Read in, Normalize and Split Data

In [15]:
target_variable = 'Strength'

In [16]:
df_cement = pd.read_csv('concrete_data.csv')

print(sorted(list(df_cement)))

['Age', 'Blast Furnace Slag', 'Cement', 'Coarse Aggregate', 'Fine Aggregate', 'Fly Ash', 'Strength', 'Superplasticizer', 'Water']


In [17]:

def normalize(data_to_normalize):
    data_to_normalize_scaled = copy.copy(data_to_normalize)
    for feature in data_to_normalize.columns:
        mean_value_of_feature = np.mean(data_to_normalize[feature].values)
        std_dev_value_of_feature = np.std(data_to_normalize[feature].values)

        data_to_normalize_scaled[feature] = (data_to_normalize[feature] - mean_value_of_feature) / std_dev_value_of_feature

    return data_to_normalize_scaled

In [18]:
# Create a features and target dataframe, where df_features does not contain target variable and df_target contains only target variable
df_features = df_cement.drop(target_variable, axis=1)

df_target = df_cement[target_variable]

df_features_normalized = normalize(data_to_normalize=df_features)
df_features_normalized

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.477915,-0.856888,-0.847144,-0.916764,-0.620448,0.863154,-1.217670,-0.279733
1,2.477915,-0.856888,-0.847144,-0.916764,-0.620448,1.056164,-1.217670,-0.279733
2,0.491425,0.795526,-0.847144,2.175461,-1.039143,-0.526517,-2.240917,3.553066
3,0.491425,0.795526,-0.847144,2.175461,-1.039143,-0.526517,-2.240917,5.057677
4,-0.790459,0.678408,-0.847144,0.488793,-1.039143,0.070527,0.647884,4.978487
...,...,...,...,...,...,...,...,...
1025,-0.045645,0.488235,0.564545,-0.092171,0.451410,-1.323005,-0.065893,-0.279733
1026,0.392819,-0.856888,0.960068,0.676200,0.702626,-1.994680,0.496893,-0.279733
1027,-1.270088,0.759579,0.850635,0.521589,-0.017528,-1.036064,0.080107,-0.279733
1028,-1.168610,1.308065,-0.847144,-0.279579,0.853356,0.214641,0.191166,-0.279733


In [19]:
#Obtain split data in dataframe format, by sending df_features and df_target to helper function
df_features_train, df_features_test, df_target_train, df_target_test = train_test_split(df_features_normalized, df_target, test_size=0.3)

# Build Keras Model

In [20]:
input_shape = (df_features_train.shape[1],)

## Create Model Architecture

In [21]:
model = Sequential()
#Three hidden layer, 
model.add(Dense(10, input_shape=input_shape, activation="relu"))
model.add(Dense(10, activation="relu"))
model.add(Dense(10, activation="relu"))
model.add(Dense(1))

# Compile and Fit Model

In [22]:
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

In [23]:
model.fit(df_features_train, df_target_train, epochs=100)

Epoch 1/100
23/23 [==============================] - 0s 568us/step - loss: 1589.9023 - mse: 1589.9023
Epoch 2/100
23/23 [==============================] - 0s 527us/step - loss: 1566.0428 - mse: 1566.0428
Epoch 3/100
23/23 [==============================] - 0s 511us/step - loss: 1528.9728 - mse: 1528.9728
Epoch 4/100
23/23 [==============================] - 0s 485us/step - loss: 1473.8074 - mse: 1473.8074
Epoch 5/100
23/23 [==============================] - 0s 465us/step - loss: 1389.5898 - mse: 1389.5898
Epoch 6/100
23/23 [==============================] - 0s 509us/step - loss: 1269.7316 - mse: 1269.7316
Epoch 7/100
23/23 [==============================] - 0s 464us/step - loss: 1110.0802 - mse: 1110.0802
Epoch 8/100
23/23 [==============================] - 0s 456us/step - loss: 912.1818 - mse: 912.1818
Epoch 9/100
23/23 [==============================] - 0s 492us/step - loss: 687.6751 - mse: 687.6751
Epoch 10/100
23/23 [==============================] - 0s 500us/step - loss: 472.2189 -

In [24]:
results = model.evaluate(x=df_features_test, y=df_target_test, return_dict=True)
print(results['mse'])

10/10 [==============================] - 0s 574us/step - loss: 81.4399 - mse: 81.4399
81.43988037109375


# Retrain Model Split and Train 50 Times

In [25]:
ls_mse_values = []

for iteration in range(0, 50):
    #Obtain split data in dataframe format, by sending df_features and df_target to helper function
    df_features_train, df_features_test, df_target_train, df_target_test = train_test_split(df_features, df_target, test_size=0.3)

    # Create a features and target dataframe, where df_features does not contain target variable and df_target contains only target variable
    df_features = df_cement.drop(target_variable, axis=1)

    df_target = df_cement[target_variable]

    df_features_normalized = normalize(data_to_normalize=df_features)

    #Obtain split data in dataframe format, by sending df_features and df_target to helper function
    df_features_train, df_features_test, df_target_train, df_target_test = train_test_split(df_features, df_target, test_size=0.3)

    input_shape = (df_features_train.shape[1],)

    #Create Model
    model = Sequential()
    #Three hidden layers, 
    model.add(Dense(10, input_shape=input_shape, activation="relu"))
    model.add(Dense(10, activation="relu"))
    model.add(Dense(10, activation="relu"))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mse', metrics=['mse'])

    model.fit(df_features_train, df_target_train, epochs=100, verbose=0)

    results = model.evaluate(x=df_features_test, y=df_target_test, return_dict=True)

    ls_mse_values.append(results['mse'])

10/10 [==============================] - 0s 519us/step - loss: 61.1283 - mse: 61.1283


## Get Mean and Standard Deviations of 50 Runs

In [27]:
avg_mse = np.mean(np.array(ls_mse_values))
std_dev_mse = np.std(np.array(ls_mse_values))

print("For the 50 runs, the mean MSE is 344.12 with a standard deviation of 361.04")
print("For the 50 runs, the mean MSE is 293.03 with a standard deviation of 374.86")
print("For the 50 runs on normalzied data with 100 epochs, the mean MSE is 161.97 with a standard deviation of 89.87")
print("For the 50 runs on normalzied data, with 3 hidden layers and 100 epochs, the mean MSE is {:.2f} with a standard deviation of {:.2f}".format(avg_mse, std_dev_mse))

For the 50 runs, the mean MSE is 344.12 with a standard deviation of 361.04
For the 50 runs, the mean MSE is 293.03 with a standard deviation of 374.86
For the 50 runs on normalzied data with 100 epochs, the mean MSE is 161.97 with a standard deviation of 89.87
For the 50 runs on normalzied data, with 3 hidden layers and 100 epochs, the mean MSE is 84.49 with a standard deviation of 33.73
